# Calculate Partial Derivatives of Phi to O2,T,S at O2=O2_bar, T=T_bar, S=S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']

In [3]:
#varr = 'O2'
#varr1 = 'O2'
#varr = 'AOU'
#varr1 = 'O2'
#varr = 'TEMP'
#varr1 = 'T'
varr = 'SALT'
varr1 = 'S'

### the partial derivatives

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_dPhid{varr1}_ens_mean_yearly.nc'
ds = xr.open_dataset(filename)

## dT, dS, dO2, dAOU

In [5]:
filename = f'{path}/DPLE_driftcorrected_d{varr}_ens_mean_yearly.nc'
ds0 = xr.open_dataset(filename)
dvar = ds0[f'd{varr}']

## calculate Phi components

In [7]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [8]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product

dsg = xr.Dataset()
for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    if varr == 'O2':
        dsg[f'Phi_{varr}_{nrow*i+j}'] =  ds[f'dPhidO2_{nrow*i+j}'] * dvar #dEodT=0.022
        print('Phi O2 done')
    elif varr == 'AOU':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = -ds[f'dPhidO2_{nrow*i+j}'] * dvar
        print('Phi AOU done')
    elif varr == 'TEMP':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = ds[f'dPhidT_{nrow*i+j}'] * dvar  #dEodT=0.022
        print('Phi T done')
    elif varr == 'SALT':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = ds[f'dPhidS_{nrow*i+j}'] * dvar  #dEodT=0.022
        print('Phi S done')

10 -0.2
Phi S done
10 -0.1
Phi S done
10 0.0
Phi S done
10 0.10000000000000003
Phi S done
10 0.2
Phi S done
10 0.3
Phi S done
10 0.4000000000000001
Phi S done
10 0.5
Phi S done
10 0.6000000000000001
Phi S done
10 0.7
Phi S done
10 0.8
Phi S done
10 0.9000000000000001
Phi S done
10 1.0000000000000002
Phi S done
CPU times: user 14 s, sys: 36 s, total: 50 s
Wall time: 3min 51s


In [9]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsg.load()

<xarray.Dataset>
Dimensions:      (year: 10, z_t: 35, nlat: 140, nlon: 195, Y: 64)
Coordinates:
  * year         (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t          (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * nlat         (nlat) int64 0 1 2 3 4 5 6 7 ... 133 134 135 136 137 138 139
  * nlon         (nlon) int64 0 1 2 3 4 5 6 7 ... 188 189 190 191 192 193 194
  * Y            (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
Data variables: (12/13)
    Phi_SALT_0   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_1   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_2   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_3   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_4   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_5   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    ...           ...
    Phi_SALT_7   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_8   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_9   (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_10  (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_11  (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_12  (year, z_t, nlat, nlon, Y) float64 nan nan nan ... nan nan nan

In [10]:
dsg.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_components_{varr}_ens_mean_yearly.nc', mode='w')